In [1]:
class EOFError(Exception):
    pass

class ConnectionBase:
    def __init__(self, connection):
        self.connection = connection
        self.file = connection.makefile('rb')
        
    def send(self, command):
        line = command + "\n"
        data = line.encode()
        self.connection.send(data)
        
    def receive(self):
        line = self.file.readline()
        if not line:
            raise EOFError('Connection closed')
        
        return line[:-1].decode()

In [4]:
import random

WARMER = '더따뜻함'
COLDER = '더차가움'
UNSURE = '잘모르겠음'
CORRECT = '맞음'


class UnknownCommandError(Exception):
    pass

class Session(ConnectionBase):
    def __init__(self, *args):
        super().__init__(*args)
        self._clear_state(None, None)

    def _clear_state(self, lower, upper):
        self.lower = lower
        self.upper = upper
        self.secret = None
        self.guesses = []
        
    def loop(self):
        while command := self.receive(): # 클라이언트로부터 얻는 매시지
            parts = command.split(' ')
            if parts[0] == 'PARAMS':
                self.set_params(parts)
            elif parts[0] == 'NUMBER':
                self.send_number()
            elif parts[0] == 'REPORT':
                self.receive_report(parts)
            else:
                raise UnknownCommandError(command)
            
    def set_params(self, parts): # 상한과 하한을 정한다
        assert len(parts) == 3
        lower = int(parts[1])
        upper = int(parts[2])
        self._clear_state(lower, upper)

    def next_guess(self):
        if self.secret is not None:
            return self.secret

        while True:
            guess = random.randint(self.lower, self.upper)
            if guess not in self.guesses:
                return guess

    def send_number(self):
        guess = self.next_guess()
        self.guesses.append(guess)
        self.send(format(guess))
        
    def receive_report(self, parts):
        assert len(parts) == 2
        decision = parts[1] # 선택
        last = self.guesses[-1]
        if decision == CORRECT: # 맞으면
            self.secret = last # 세션의 상태를 바꿈

        print(f'서버: {last}는 {decision}')

In [5]:
import contextlib
import math

class Client(ConnectionBase):
    def __init__(self, *args):
        super().__init__(*args)
        self._clear_state()

    def _clear_state(self):
        self.secret = None
        self.last_distance = None
        
    @contextlib.contextmanager
    def session(self, lower, upper, secret):
        print(f'\n{lower}와 {upper} 사이의 숫자를 맞춰보세요!'
              f' 쉿! 그 숫자는 {secret} 입니다.')
        self.secret = secret
        self.send(f'PARAMS {lower} {upper}')
        try:
            yield
        finally:
            self._clear_state()
            self.send('PARAMS 0 -1')
            
    def request_numbers(self, count):
        for _ in range(count):
            self.send('NUMBER')
            data = self.receive()
            yield int(data)
            if self.last_distance == 0:
                return
            
            
    def report_outcome(self, number):
        new_distance = math.fabs(number - self.secret)
        decision = UNSURE

        if new_distance == 0:
            decision = CORRECT
        elif self.last_distance is None:
            pass
        elif new_distance < self.last_distance:
            decision = WARMER
        elif new_distance > self.last_distance:
            decision = COLDER

        self.last_distance = new_distance

        self.send(f'REPORT {decision}')
        return decision

In [6]:
import socket
from threading import Thread


def handle_connection(connection):
    with connection:
        session = Session(connection)
        try:
            session.loop()
        except EOFError:
            pass

In [7]:
def run_server(address):
    with socket.socket() as listener:
        listener.bind(address)
        listener.listen()
        while True:
            connection, _ = listener.accept()
            thread = Thread(target=handle_connection,
                            args=(connection,),
                            daemon=True)
            thread.start()

In [8]:
def run_client(address):
    with socket.create_connection(address) as connection:
        client = Client(connection)

        with client.session(1, 5, 3):
            results = [(x, client.report_outcome(x))
                       for x in client.request_numbers(5)]

        with client.session(10, 15, 12):
            for number in client.request_numbers(5):
                outcome = client.report_outcome(number)
                results.append((number, outcome))

    return results

In [9]:
def main():
    address = ('127.0.0.1', 1234)
    server_thread = Thread(
        target=run_server, args=(address,), daemon=True)
    server_thread.start()

    results = run_client(address)
    for number, outcome in results:
        print(f'클라이언트: {number}는 {outcome}')

main()


1와 5 사이의 숫자를 맞춰보세요! 쉿! 그 숫자는 3 입니다.
서버: 1는 잘모르겠음
서버: 5는 잘모르겠음
서버: 2는 더따뜻함
서버: 4는 잘모르겠음

10와 15 사이의 숫자를 맞춰보세요! 쉿! 그 숫자는 12 입니다.
서버: 3는 맞음
클라이언트: 1는 잘모르겠음
클라이언트: 5는 잘모르겠음
클라이언트: 2는 더따뜻함
클라이언트: 4는 잘모르겠음
클라이언트: 3는 맞음
클라이언트: 12는 맞음
서버: 12는 맞음
